In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Change directory to a specific folder in Google Drive
os.chdir('/content/drive/MyDrive/FPGA Acce')

# Verify current working directory
print("Current working directory:", os.getcwd())

Mounted at /content/drive
Current working directory: /content/drive/MyDrive/FPGA Acce


In [ ]:
!pwd
!ls

/content/drive/MyDrive/FPGA Acce
biasValues.h	     __pycache__	w_b_final
genTestData.py	     sigContent.mif	weightsandbiases_final.txt
genWegitsAndBias.py  sigContent.mif_32	WeightsAndBiases.txt
mnist_loader.py      testData		weightValues.h
mnist.pkl.gz	     testData_32
network2.py	     trainNN.ipynb


In [ ]:
#Sigmoid Generation

# -*- coding: utf-8 -*-
"""
Created on Mon Oct  6 19:23:47 2025

@author: Rhodz
"""


import math

def genSigContent(dataWidth,sigmoidSize,weightIntSize,inputIntSize):
    f = open("sigContent.mif","w")
    fractBits = sigmoidSize-(weightIntSize+inputIntSize)
    if fractBits < 0: #Sigmoid size is smaller the integer part of the MAC operation
        fractBits = 0
    x = -2**(weightIntSize+inputIntSize-1)#Smallest input going to the Sigmoid LUT from the neuron
    for i in range(0,2**sigmoidSize):
        y = sigmoid(x)
        z = DtoB(y,dataWidth,dataWidth-inputIntSize)
        print(y,z)
        f.write(z+'\n')
        x=x+(2**-fractBits)
    f.close()

def DtoB(num,dataWidth,fracBits):#funtion for converting into two's complement format
    if num >= 0:
        num = num * (2**fracBits)
        num = int(num)
        e = bin(num)[2:]
    else:
        num = -num
        num = num * (2**fracBits)#number of fractional bits
        num = int(num)
        if num == 0:
            d = 0
        else:
            d = 2**dataWidth - num
        e = bin(d)[2:]
    return e


def sigmoid(x):
    try:
        return 1 / (1+math.exp(-x))#for x less than -1023 will give value error
    except:
        return 0


if __name__ == "__main__":
    genSigContent(dataWidth=16,sigmoidSize=10,weightIntSize=4,inputIntSize=1)
#output sigmoid.mif is loaded for the sigmoid ROM

1.12535162055095e-07 0
1.1610741114742279e-07 0
1.1979305557162662e-07 0
1.2359569488020953e-07 0
1.275190428876251e-07 0
1.3156693129733423e-07 0
1.3574331344399593e-07 0
1.4005226815444813e-07 0
1.4449800373124837e-07 0
1.4908486206266502e-07 0
1.5381732286313315e-07 0
1.587000080483163e-07 0
1.63737686249047e-07 0
1.6893527746855403e-07 0
1.7429785788752586e-07 0
1.7983066482170178e-07 0
1.8553910183683314e-07 0
1.914287440260102e-07 0
1.9750534345450816e-07 0
2.0377483477747004e-07 0
2.1024334103591293e-07 0
2.1691717963671816e-07 0
2.238028685224453e-07 0
2.3090713253699577e-07 0
2.38236909993343e-07 0
2.4579935944974304e-07 0
2.5360186670104374e-07 0
2.6165205199192004e-07 0
2.6995777745908016e-07 0
2.7852715480971087e-07 0
2.8736855324366057e-07 0
2.964906076270974e-07 0
3.059022269256247e-07 0
3.156126029050898e-07 0
3.256312191085833e-07 0
3.359678601183963e-07 0
3.466326211119807e-07 0
3.576359177212449e-07 0
3.6898849620481354e-07 0
3.8070144394318623e-07 0
3.927862002670442

In [ ]:
#Training Modle via tensorflow keras
import tensorflow as tf
import json

mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train=tf.keras.utils.normalize(x_train,axis=1)
x_test=tf.keras.utils.normalize(x_test,axis=1)

model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(30,activation=tf.nn.sigmoid))
model.add(tf.keras.layers.Dense(30,activation=tf.nn.sigmoid))
model.add(tf.keras.layers.Dense(10,activation=tf.nn.sigmoid))
model.add(tf.keras.layers.Dense(10,activation=tf.nn.sigmoid))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=20)

val_loss,val_acc=model.evaluate(x_test,y_test)
print(val_loss,val_acc)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.3481 - loss: 1.9175
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8132 - loss: 0.7814
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8957 - loss: 0.4745
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9232 - loss: 0.3219
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9379 - loss: 0.2467
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9480 - loss: 0.2044
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9547 - loss: 0.1746
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9570 - loss: 0.1607
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9630 - loss: 0.1402
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9655 - loss: 0.1308
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0

In [ ]:
#Save weights to a .txt file
weightList = []
biasList = []
for i in range(1,len(model.layers)):
    weights = model.layers[i].get_weights()[0]
    weightList.append((weights.T).tolist())
    bias = [[float(b)] for b in model.layers[i].get_weights()[1]]
    biasList.append(bias)

data = {"weights": weightList,"biases":biasList}
f = open('weightsandbiases_final.txt', "w") #txt file output
json.dump(data, f)
f.close()

In [ ]:
!ls

biasValues.h	     __pycache__	w_b_32
genTestData.py	     sigContent.mif	w_b_final
genWegitsAndBias.py  sigContent.mif_32	weightsandbiases_final.txt
mnist_loader.py      testData		WeightsAndBiases.txt
mnist.pkl.gz	     testData_32	weightValues.h
network2.py	     trainNN.ipynb


In [ ]:
#Weights and Biases conversion to .if file
import json

dataWidth = 16
dataIntWidth = 1
weightIntWidth = 4
inputFile = "weightsandbiases_final.txt" # output of the previous cell
dataFracWidth = dataWidth-dataIntWidth
weightFracWidth = dataWidth-weightIntWidth
biasIntWidth = dataIntWidth+weightIntWidth
biasFracWidth = dataWidth-biasIntWidth
outputPath = "./w_b/" # location of weights and biases files. Make this folder manually
headerPath = "./"

def DtoB(num,dataWidth,fracBits):						#funtion for converting into two's complement format
	if num >= 0:
		num = num * (2**fracBits)
		num = int(num)
		d = num
	else:
		num = -num
		num = num * (2**fracBits)		#number of fractional bits
		num = int(num)
		if num == 0:
			d = 0
		else:
			d = 2**dataWidth - num
	return d

def genWaitAndBias(dataWidth,weightFracWidth,biasFracWidth,inputFile):
	weightIntWidth = dataWidth-weightFracWidth
	biasIntWidth = dataWidth-biasFracWidth
	myDataFile = open(inputFile,"r")
	weightHeaderFile = open(headerPath+"weightValues.h","w")
	myData = myDataFile.read()
	myDict = json.loads(myData)
	myWeights = myDict['weights']
	myBiases = myDict['biases']
	weightHeaderFile.write("int weightValues[]={")
	for layer in range(0,len(myWeights)):
		for neuron in range(0,len(myWeights[layer])):
			fi = 'w_'+str(layer+1)+'_'+str(neuron)+'.mif'
			f = open(outputPath+fi,'w')
			for weight in range(0,len(myWeights[layer][neuron])):
				if 'e' in str(myWeights[layer][neuron][weight]):
					p = '0'
				else:
					if myWeights[layer][neuron][weight] > 2**(weightIntWidth-1):
						myWeights[layer][neuron][weight] = 2**(weightIntWidth-1)-2**(-weightFracWidth)
					elif myWeights[layer][neuron][weight] < -2**(weightIntWidth-1):
						myWeights[layer][neuron][weight] = -2**(weightIntWidth-1)
					wInDec = DtoB(myWeights[layer][neuron][weight],dataWidth,weightFracWidth)
					p = bin(wInDec)[2:]
				f.write(p+'\n')
				weightHeaderFile.write(str(wInDec)+',')
			f.close()
	weightHeaderFile.write('0};\n')
	weightHeaderFile.close()

	biasHeaderFile = open(headerPath+"biasValues.h","w")
	biasHeaderFile.write("int biasValues[]={")
	for layer in range(0,len(myBiases)):
		for neuron in range(0,len(myBiases[layer])):
			fi = 'b_'+str(layer+1)+'_'+str(neuron)+'.mif'
			p = myBiases[layer][neuron][0]
			if 'e' in str(p): #To remove very small values with exponents
				res = '0'
			else:
				if p > 2**(biasIntWidth-1):
					p = 2**(biasIntWidth-1)-2**(-biasFracWidth)
				elif p < -2**(biasIntWidth-1):
					p = -2**(biasIntWidth-1)
				bInDec = DtoB(p,dataWidth,biasFracWidth)
				res = bin(bInDec)[2:]
			f = open(outputPath+fi,'w')
			f.write(res)
			biasHeaderFile.write(str(bInDec)+',')
			f.close()
	biasHeaderFile.write('0};\n')
	biasHeaderFile.close()

if __name__ == "__main__":
	genWaitAndBias(dataWidth,weightFracWidth,biasFracWidth,inputFile)

In [ ]:
!ls

biasValues.h	     __pycache__	w_b
genTestData.py	     sigContent.mif	w_b_32
genWegitsAndBias.py  sigContent.mif_32	w_b_final
mnist_loader.py      testData		weightsandbiases_final.txt
mnist.pkl.gz	     testData_32	WeightsAndBiases.txt
network2.py	     trainNN.ipynb	weightValues.h


In [ ]:
#Generating Test Data
import sys

outputPath = "./testData/" #Manually create this folder
headerFilePath = "./testData/"

try:
    import cPickle as pickle
except:
    import pickle
import gzip
import numpy as np

dataWidth = 16                    #specify the number of bits in test data
IntSize = 1 #Number of bits of integer portion including sign bit

try:
    testDataNum = int(sys.argv[1])
except:
    testDataNum = 3

def DtoB(num,dataWidth,fracBits):                        #funtion for converting into two's complement format
    if num >= 0:
        num = num * (2**fracBits)
        d = int(num)
    else:
        num = -num
        num = num * (2**fracBits)        #number of fractional bits
        num = int(num)
        if num == 0:
            d = 0
        else:
            d = 2**dataWidth - num
    return d


def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')         #change this location to the resiprositry where MNIST dataset sits
    try:
        training_data, validation_data, test_data = pickle.load(f,encoding='latin1')
    except:
        training_data, validation_data, test_data = pickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def genTestData(dataWidth,IntSize,testDataNum):
    dataHeaderFile = open(headerFilePath+"dataValues.h","w")
    dataHeaderFile.write("int dataValues[]={")
    tr_d, va_d, te_d = load_data()
    test_inputs = [np.reshape(x, (1, 784)) for x in te_d[0]]
    x = len(test_inputs[0][0])
    d=dataWidth-IntSize
    count = 0
    fileName = 'test_data.txt'
    f = open(outputPath+fileName,'w')
    fileName = 'visual_data'+str(te_d[1][testDataNum])+'.txt'
    g = open(outputPath+fileName,'w')
    k = open('testData.txt','w')
    for i in range(0,x):
        k.write(str(test_inputs[testDataNum][0][i])+',')
        dInDec = DtoB(test_inputs[testDataNum][0][i],dataWidth,d)
        myData = bin(dInDec)[2:]
        dataHeaderFile.write(str(dInDec)+',')
        f.write(myData+'\n')
        if test_inputs[testDataNum][0][i]>0:
            g.write(str(1)+' ')
        else:
            g.write(str(0)+' ')
        count += 1
        if count%28 == 0:
            g.write('\n')
    k.close()
    g.close()
    f.close()
    dataHeaderFile.write('0};\n')
    dataHeaderFile.write('int result='+str(te_d[1][testDataNum])+';\n')
    dataHeaderFile.close()


def genAllTestData(dataWidth,IntSize):
    tr_d, va_d, te_d = load_data()
    test_inputs = [np.reshape(x, (1, 784)) for x in te_d[0]]
    x = len(test_inputs[0][0])
    d=dataWidth-IntSize
    for i in range(len(test_inputs)):
        if i < 10:
            ext = "000"+str(i)
        elif i < 100:
            ext = "00"+str(i)
        elif i < 1000:
            ext = "0"+str(i)
        else:
            ext = str(i)
        fileName = 'test_data_'+ext+'.txt'
        f = open(outputPath+fileName,'w')
        for j in range(0,x):
            dInDec = DtoB(test_inputs[i][0][j],dataWidth,d)
            myData = bin(dInDec)[2:]
            f.write(myData+'\n')
        f.write(bin(DtoB((te_d[1][i]),dataWidth,0))[2:])
        f.close()



if __name__ == "__main__":
    #genTestData(dataWidth,IntSize,testDataNum=1)
    genAllTestData(dataWidth,IntSize)

In [ ]:
!ls

biasValues.h	     network2.py    weightsandbiases_1.txt
gen_nn.py	     __pycache__    weightsandbiases.txt
genTestData.py	     testData	    WeightsAndBiases.txt
genWegitsAndBias.py  trainNN.ipynb  weightValues.h
__init__.py	     utils.py	    xilinxUtils.py
mnist_loader.py      w_b	    zynet.py
mnist.pkl.gz	     w_b_1
